<a name="top"></a>
<div style="width:1000 px">

<div style="float:right; width:98 px; height:98px;">
<img src="https://raw.githubusercontent.com/Unidata/MetPy/master/src/metpy/plots/_static/unidata_150x150.png" alt="Unidata Logo" style="height: 98px;">
</div>

<h1>Rossby Waves</h1>
<h3>Unidata AMS 2021 Student Conference</h3>

<div style="clear:both"></div>
</div>

---

If you'd want a starting point for the Rossby Wave Analysis project suggestion, feel free to take a look through this notebook for ideas! It goes over some of the basics of Rossby Wave theory (but not quite enough to teach you everything you'd learn in an actual dynamics course), as well as some approaches to analyze the data. This notebook also uses [scipy](https://docs.scipy.org/doc/scipy-1.6.0/reference/) in several places, so feel free to click on the links to learn more about the functionality available in that library.

<div style="float:right; width:250 px"><img src="../../instructors/images/rossby_wave_screenshot.png" alt="Cyclically smoothing heights" style="height: 300px;"></div>


### Focuses

- Obtain GFS data from which we can perform our analysis
- Obtain calculated paramters:
    - Smoothed 500 hPa heights 
    - Zonal-average zonal winds at 500 hPa and the 150-300 hPa layer maximum
    - Wave maxima/minima, and from these:
        - Wave number
        - Average wave amplitude
        - Average wave speed
- Compare the observed data against our expected results from basic Rossby Wave theory.
- Make Hovmöller diagrams of our height data, to visualize wave evolution over the period

### Objectives

1. [Source Data Retrieval](#1.-Source-Data-Retrieval)
1. [Parameter Derivation](#2.-Parameter-Derivation)
1. [Statistical Comparison Against Theory](#3.-Statistical-Comparison-Against-Theory)
1. [Hovmöller Diagrams](4.-Hovmöller-Diagrams)

---

In [ ]:
# Imports
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt

from scipy.signal import savgol_filter
from scipy.stats import linregress
from datetime import datetime, timedelta
from metpy.units import units
from metpy.constants import earth_avg_radius, earth_avg_angular_vel

<a href="#top">Top</a>

---

## 1. Source Data Retrieval

Here, we use some already-parsed sample data that just gives us zonal winds and geopotential heights along two latitude circles. *How would you use xarray remote data access and indexing to create your own similar dataset?*

In [ ]:
data = xr.open_dataset('../../instructors/practice_files/sample_rossby_wave_data.nc')
data

<a href="#top">Top</a>

---

## 2. Parameter Derivation

This section is (deliberately) rough! Think of how you could use what you've learned to improve upon the code in this section, or implement other methods to derive these wave parameters.

### Smoothed 500 hPa Heights

Smooth the 1-D height data with [a Savitzky-Golay filter from scipy](https://docs.scipy.org/doc/scipy-1.6.0/reference/generated/scipy.signal.savgol_filter.html).

In [ ]:
# Rename the height field
heights = data['Geopotential_height_isobaric'].rename('heights')

# Smooth the source data and re-insert into a new xarray DataArray
heights_smoothed = xr.DataArray(
    savgol_filter(heights, 39, 2, mode="wrap"),
    coords=heights.coords,
    attrs=heights.attrs,
    name='heights_smoothed'
)

In [ ]:
# Compare the raw and smoothed data at 500 hPa (using NH, initial time as example)
heights.sel(isobaric=50000., lat=50).isel(time=0).plot(color='orange')
heights_smoothed.sel(isobaric=50000., lat=50).isel(time=0).plot(color='blue')

### Zonal Winds

In [ ]:
# Get the 500 hPa, and 150-300 hPa layer-max zonal-average zonal wind
zonal_winds = data['u-component_of_wind_isobaric'].mean(dim='lon')
zonal_wind_500 = zonal_winds.sel(isobaric=50000.)
zonal_wind_upper = zonal_winds.where(zonal_winds.isobaric <= 30000, drop=True).max(dim='isobaric')

In [ ]:
# Plot over time (for example, in SH)
zonal_wind_500.sel(lat=-50).plot(color='red')
zonal_wind_upper.sel(lat=-50).plot(color='blue')

<a href="#top">Top</a>

---

### Wave Properties

Don't get scared off by this section! It is trying to derive wave amplitude, number, and speed through analysis of the maxima and minima. Could you think of other ways to obtain these same parameters from the smoothed height data?

In [ ]:
# Define some helpful functions
def get_minima_indicies(array):
    return (np.diff(np.sign(np.diff(array))) > 0).nonzero()[0] + 1


def get_maxima_indicies(array):
    return (np.diff(np.sign(np.diff(array))) < 0).nonzero()[0] + 1


def get_increments(previous_extrema, current_extrema):
    # Make sure these are cast as lists
    previous_extrema = list(previous_extrema)
    current_extrema = list(current_extrema)

    # Adjust for overflow
    current_extrema = cycle_adjust(previous_extrema, current_extrema)

    increments = []
    for lon in current_extrema:
        # Find closest extrema in previous, and assume continuation
        prev_lon = min(previous_extrema, key=lambda x:abs(x-lon))
        increments.append(lon - prev_lon)

    return np.array(increments)


def cycle_adjust(previous_extrema, current_extrema):
    # Bump up an overflowed element
    if min(current_extrema) < min(previous_extrema)/2:
        # If the the new lowest element is closer to zero than it is to the previous lowest,
        # assume it is an overflow
        overflowed = current_extrema.pop(0)
        current_extrema.append(overflowed + 360.)
        
    return current_extrema


def reject_outliers(data):
    d = data - np.median(data)
    iqr = np.percentile(d, 75) - np.percentile(d, 25)
    iqr = iqr if iqr > 0 else 1.

    return data[np.abs(d) < 1.5 * iqr]


def fraction_of_day(td):
    return td / np.timedelta64(1, 'D')

In [ ]:
# Variables for storage of analyzed values (lists of lists)
minima_full = [[] for lat in heights_smoothed.lat]
maxima_full = [[] for lat in heights_smoothed.lat]
wave_number_full = [[] for lat in heights_smoothed.lat]
wave_amplitude_full = [[] for lat in heights_smoothed.lat]
wave_speed_full = [[] for lat in heights_smoothed.lat]

# Loop over hemispheres and times to compute wave properties (extrema, wave number, wave amplitude, wave speed)
for j in range(0, len(heights_smoothed.lat)):
    
    lat = heights_smoothed.lat[j]
    
    # Get this particular latitude's storage variables
    minima = minima_full[j]
    maxima = maxima_full[j]
    wave_number = wave_number_full[j]
    wave_amplitude = wave_amplitude_full[j]
    wave_speed = wave_speed_full[j]

    for i in range(0, len(heights_smoothed.time)):
        # Filter out this time's height data
        current_heights = heights_smoothed.sel(isobaric=50000., lat=lat).isel(time=i)
        
        # Get the minima and maxima
        minima_indicies = get_minima_indicies(current_heights)
        maxima_indicies = get_maxima_indicies(current_heights)
        minima.append(current_heights.lon[minima_indicies].values)
        maxima.append(current_heights.lon[maxima_indicies].values)
        
        # Calculate Wave Number
        k = max(len(minima_indicies), len(maxima_indicies))
        wave_number.append(k)
        
        # Calculate Amplitude
        trough_avg = current_heights[minima_indicies].values.mean()
        ridge_avg = current_heights[maxima_indicies].values.mean()
        a = (ridge_avg-trough_avg)/2.
        wave_amplitude.append(a)
        
        # Calculate Wave Speed
        if i == 0:
            # First time, cannot compute one-sided backwards difference
            c = np.nan
        else:
            # Otherwise, calculate speed (in deg lon/day)
            previous_minima = minima[i-1]
            previous_maxima = maxima[i-1]
            current_minima = minima[i]
            current_maxima = maxima[i]
            
            time_diff = heights_smoothed.time[i] - heights_smoothed.time[i-1]
            
            minima_increments = get_increments(previous_minima, current_minima)
            maxima_increments = get_increments(previous_maxima, current_maxima)
            lon_increments = np.concatenate([minima_increments, maxima_increments])
            
            c = reject_outliers(lon_increments).mean() / fraction_of_day(time_diff.values)
        
        wave_speed.append(c)
        
# Get our variables back into nice DataArrays
wave_property_coords = [('lat', heights_smoothed.lat), ('time', heights_smoothed.time)]

wave_number_raw = xr.DataArray(wave_number_full, coords=wave_property_coords, name='wave_number')
wave_amplitude = xr.DataArray(wave_amplitude_full, coords=wave_property_coords, name='wave_amplitude')
wave_speed = xr.DataArray(wave_speed_full, coords=wave_property_coords, name='wave_speed')

# Set unit metadata
wave_number_raw.attrs['units'] = 'earth_radius**-1'
wave_amplitude.attrs['units'] = 'm'
wave_speed.attrs['units'] = 'degrees_east day**-1'

In [ ]:
# Now, for example, see how the wave speed changed over time (NH)
wave_speed.sel(lat=50).plot()

<a href="#top">Top</a>

---

## 3. Statistical Comparison Against Theory

For the barotropic long-waves of Rossby Wave Theory, we expect the following phase speed:

$$c_x = u - \frac{\beta}{k^2 + l^2},$$

where $u$ is background zonal flow, $\beta = \frac{\partial f}{\partial y}$, and $k$ and $l$ are zonal and meridional wave numbers, respectively. We also expect that wave patterns with higher wave numbers will have lower amplitudes. And so, below we will investigate the degree to which our observations fit our theory by looking at the following correlations:

- Wave Speed vs. Zonal Wind
- Wave Speed vs. Wave Number
- Wave Amplitude vs. Wave Number

### Correlations

To [perform the linear regressions, we use scipy](https://docs.scipy.org/doc/scipy-1.6.0/reference/generated/scipy.stats.linregress.html).

*Unit Conversions*

- Convert `wave_number` from earth_radius<sup>-1</sup> to m<sup>-1</sup>
- Convert `wave_speed` from deg_lon day<sup>-1</sup> to m s<sup>-1</sup>

In [ ]:
lat = 50 * units('degrees')

# Wave number conversion factor
scale_factor = 1 / earth_avg_radius
wave_number = wave_number_raw * scale_factor
wave_number.attrs['units'] = 'm^-1'

# Wave speed conversion factor
scale_factor = (units('degrees day^-1') * 2 * np.pi * np.cos(lat) * earth_avg_radius / (360 * units('degrees'))).to('m s^-1')
# Make sure this runs only once
if wave_speed.attrs['units'] == 'degrees_east day^-1':
    wave_speed = wave_speed * scale_factor
    wave_speed.attrs['units'] = 'm s^-1'

**Wave Speed vs. Zonal Wind**

Here, we would expect to see a roughly linear relationship, with slope near 1, but variability in the y-intercept due to the (presently unaccounted for) wave number.

In [ ]:
# Define four different subsets of our data to consider
configs = [
    {
        'hemisphere': 1,
        'hemisphere_label': 'Northern Hemisphere',
        'wind': zonal_wind_500,
        'wind_label': '500 hPa Zonal Wind'
    },
    {
        'hemisphere': 1,
        'hemisphere_label': 'Northern Hemisphere',
        'wind': zonal_wind_upper,
        'wind_label': '150-300 hPa Layer Maximum Zonal Wind'
    },
    {
        'hemisphere': -1,
        'hemisphere_label': 'Southern Hemisphere',
        'wind': zonal_wind_500,
        'wind_label': '500 hPa Zonal Wind'
    },
    {
        'hemisphere': -1,
        'hemisphere_label': 'Southern Hemisphere',
        'wind': zonal_wind_upper,
        'wind_label': '150-300 hPa Layer Maximum Zonal Wind'
    } 
]

# Loop over the configurations
for config in configs:
    # Mask the initial NaN, and calculate linear regression
    y = np.ma.masked_invalid(wave_speed.sel(lat=config['hemisphere'] * 50).values)
    x = np.ma.masked_where(np.ma.getmask(y), config['wind'].sel(lat=config['hemisphere'] * 50).values) 
    m, b, r, p, stderr = linregress(x.compressed(), y.compressed())

    # Set up plot
    ax = plt.subplot()
    ax.set_title('{} \nwith {}'.format(config['hemisphere_label'], config['wind_label']))
    ax.set_ylabel('Wave Speed (m s^-1)')
    ax.set_xlabel('{} (m s^-1)'.format(config['wind_label']))

    # Scatter Plot
    ax.scatter(x, y)

    # Regression Line
    xfill = np.array([x.min(), x.max()])
    ax.plot(xfill, m * xfill + b, color='red')

    plt.show()
    print("Slope: {}".format(m))
    print("Correlation: {}".format(r))

**Wave Speed vs. Wave Number**

Here, we first have to normalize our wave speed and wave number in order to obtain a theoretical linear relationship:

$$c_x - u = (-\beta) \frac{1}{k^2},$$

(Note that this also ignores meridional wave number, since we are primarily concerned with the zonal waves.) And so, we would expect to see a linear relationship, with slope near $-\beta$, and a y-intercept near zero.

In [ ]:
# Find beta (same in NH and SH due to symmetry)
beta = (2 * earth_avg_angular_vel * np.cos(lat) / earth_avg_radius).to('m^-1 s^-1')
beta

In [ ]:
# Prepare our variables
normalized_wave_speed = wave_speed - zonal_wind_500
linearized_wave_number = 1 / wave_number ** 2

# Define two different subsets of our data to consider
configs = [
    {
        'hemisphere': 1,
        'hemisphere_label': 'Northern Hemisphere'
    },
    {
        'hemisphere': -1,
        'hemisphere_label': 'Southern Hemisphere'
    }
]

# Loop over the configurations
for config in configs:
    # Mask the initial NaN, and calculate linear regression
    y = np.ma.masked_invalid(normalized_wave_speed.sel(lat=config['hemisphere'] * 50).values)
    x = np.ma.masked_where(np.ma.getmask(y), linearized_wave_number.sel(lat=config['hemisphere'] * 50).values) 
    m, b, r, p, stderr = linregress(x.compressed(), y.compressed())

    # Set up plot
    ax = plt.subplot()
    ax.set_title(config['hemisphere_label'])
    ax.set_ylabel('Normalized Wave Speed (m s^-1)')
    ax.set_xlabel('Square Reciprocal of Wave Number(m^2)')

    # Scatter Plot
    ax.scatter(x, y)

    # Regression Line
    xfill = np.array([x.min(), x.max()])
    ax.plot(xfill, m * xfill + b, color='red')

    plt.show()
    print("Slope: {}".format(m))
    print("Correlation: {}".format(r))

**Wave Amplitude vs. Wave Number**

This is a resonably straightforward relationship: we expect the wave amplitude to decrease as wave number increases.

In [ ]:
# Define two different subsets of our data to consider
configs = [
    {
        'hemisphere': 1,
        'hemisphere_label': 'Northern Hemisphere'
    },
    {
        'hemisphere': -1,
        'hemisphere_label': 'Southern Hemisphere'
    }
]

# Loop over the configurations
for config in configs:
    # Mask the initial NaN, and calculate linear regression
    y = wave_amplitude.sel(lat=config['hemisphere'] * 50).values
    x = wave_number_raw.sel(lat=config['hemisphere'] * 50).values
    m, b, r, p, stderr = linregress(x, y)

    # Set up plot
    ax = plt.subplot()
    ax.set_title(config['hemisphere_label'])
    ax.set_ylabel('Normalized Wave Speed (m s^-1)')
    ax.set_xlabel('Wave Number(earth_radius^-1)')

    # Scatter Plot
    ax.scatter(x, y)

    # Regression Line
    xfill = np.array([x.min(), x.max()])
    ax.plot(xfill, m * xfill + b, color='red')

    plt.show()
    print("Slope: {}".format(m))
    print("Correlation: {}".format(r))

**Conclusions**

What we found in our analysis (at least for the sample case set used here) was that most (but not all) of the trends expected by Rossby Wave Theory were observed in our data, but the regression statistics did not usually fit with their theoretical values. This mismatch indicates the limitations in our simplified barotropic theory when trying to apply it to the real atmosphere in the mid-latitudes, which is often baroclinic.

<a href="#top">Top</a>

---

## 4. Hovmöller Diagrams

xarray makes it easy to make basic Hovmöller diagrams! Just subset and use `.plot`! More advanced examples are also certainly possible by leveraging the full flexibility of matplotlib.

In [ ]:
# Northern Hemisphere
heights_smoothed.sel(isobaric=50000., lat=50).plot(yincrease=False)
plt.title('Northern Hemisphere')
plt.show()

In [ ]:
# Southern Hemisphere
heights_smoothed.sel(isobaric=50000., lat=-50).plot(yincrease=False)
plt.title('Southern Hemisphere')
plt.show()

<a href="#top">Top</a>